# Aula 4: Classificação com Redes Neurais

- Biblioteca Keras: abstração para TensorFlow
  - Topologia da rede é constuindo empilhando camadas: modelo sequencial
  - Tipos de camadas
    - Densa: conectada
    - Embedding
    - Dropout
  - Funções de ativação: relu, sigmoid
  - Hiperparametros
    - Loss: MSE
    - Optimizer: Adam
    - Metrics

# Aula 5: Implementando RN I

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

In [2]:
url = "https://raw.githubusercontent.com/AnaClaraGuerra22/PLN-LLMs-e-Gen-AI---Udemy/52804be1d038ac399f9c9b3594330a5df38f808c/ML%20e%20DL%20para%20NLP/spam.csv"

spam = pd.read_csv(url)
print(spam.shape)
spam.head()

(5572, 2)


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# target = 'category'

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(spam['Category'])

print(y)

# problema de classificação binario
# 0: ham
# 1: spam

[0 0 1 ... 0 0 0]


In [5]:
mensagens = spam['Message'].values

In [7]:
x_train, x_test, y_train, y_test = train_test_split(mensagens, y, test_size=0.3)
print("X treinamento: ", x_train)
print("\nX teste: ", x_test)
print("\nY treinamento: ", y_train)
print("\nY teste: ",y_test)

X treinamento:  ['u r subscribed 2 TEXTCOMP 250 wkly comp. 1st wk?s free question follows, subsequent wks charged@150p/msg.2 unsubscribe txt STOP 2 84128,custcare 08712405020'
 "You won't believe it but it's true. It's Incredible Txts! Reply G now to learn truly amazing things that will blow your mind. From O2FWD only 18p/txt"
 'Were trying to find a Chinese food place around here' ...
 'Welcome to UK-mobile-date this msg is FREE giving you free calling to 08719839835. Future mgs billed at 150p daily. To cancel send "go stop" to 89123'
 'YOU 07801543489 are guaranteed the latests Nokia Phone, a 40GB iPod MP3 player or a £500 prize! Txt word:COLLECT to No:83355! TC-LLC NY-USA 150p/Mt msgrcvd18+'
 "Neva mind it's ok.."]

X teste:  ['When u love someone Dont make them to love u as much as u do. But Love them so much that they dont want to be loved by anyone except you... Gud nit.'
 'My uncles in Atlanta. Wish you guys a great semester.'
 'Ur cash-balance is currently 500 pounds - to maxim

In [8]:
vetorizador = CountVectorizer()

vetorizador.fit(x_train)
x_train = vetorizador.transform(x_train)
x_test = vetorizador.transform(x_test)

print("X treinamento: ", x_train.toarray())
print("\nX teste: ", x_test.toarray())

X treinamento:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

X teste:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Aula 6: Implementando RN II

In [11]:
x_train.toarray()[0]

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
print(x_train.shape[1])

7255


In [15]:
# criando camadas

modelo = Sequential()

# units = qtd de neuronios
# primeira camada, precisa colocar a entrada
modelo.add(Dense(units=10, activation='relu', input_dim=x_train.shape[1]))

# droupout, minimizar overfitting
modelo.add(Dropout(0.1))

modelo.add(Dense(units=8, activation='relu'))

modelo.add(Dropout(0.1))
# camada de saida, units nao pode ser valor arbitrario, tem q ser equivalente a classe
# Ex: se for classe a,b,c --> units=3
# Ex: problema binario com funçao de ativaçao sigmoide, caso atual: units = 1 [0,1]
modelo.add(Dense(units=1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
modelo.compile(loss='mean_squared_error', optimizer='Adam', metrics=['accuracy'])
modelo.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 10)             │        72,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │            88 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 72,657 (283.82 KB)

 Trainable params: 72,657 (283.82 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
modelo.fit(x_train, y_train, epochs=100, batch_size=32, verbose=True, validation_data=(x_test, y_test))

Epoch 1/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8652 - loss: 0.1766 - val_accuracy: 0.8654 - val_loss: 0.0668
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8728 - loss: 0.0579 - val_accuracy: 0.8654 - val_loss: 0.0506
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8692 - loss: 0.0457 - val_accuracy: 0.9713 - val_loss: 0.0458
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9795 - loss: 0.0396 - val_accuracy: 0.9737 - val_loss: 0.0432
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9835 - loss: 0.0344 - val_accuracy: 0.9785 - val_loss: 0.0394
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9943 - loss: 0.0317 - val_accuracy: 0.9797 - val_loss: 0.0375
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9975 - loss: 0.0288 - val_accuracy: 0.9815 - val_loss: 0.0353
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9961 - loss: 0.0238 - val_acc

# Aula 7: Implementando RN III

In [18]:
loss, accuracy = modelo.evaluate(x_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9845 - loss: 0.0156
Loss:  0.01646062731742859
Accuracy:  0.9832535982131958


In [21]:
new_prevision = modelo.predict(x_test)
new_prevision

#sigmnoide retorna [0,1]
# 0: ham, 1:spam

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([[3.0535222e-28],
       [1.4967475e-13],
       [9.8377794e-01],
       ...,
       [1.2759011e-22],
       [9.8377794e-01],
       [2.0745671e-13]], dtype=float32)

In [22]:
# vira bool
prev = (new_prevision > 0.5)
prev

array([[False],
       [False],
       [ True],
       ...,
       [False],
       [ True],
       [False]])

In [23]:
cm = confusion_matrix(y_test, prev)
cm

array([[1446,    1],
       [  27,  198]])